In [1]:
import random,math,copy,timeit,operator
from sklearn import utils,preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sys import exit
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection as ms

In [2]:
class ants(object):
    def __init__(self,gens=None,mse=None):
        self.gens=gens
        self.mse=mse

In [3]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [4]:
def k_search(n,dim,flist,pheromone):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
        for j,x in enumerate(gen):
            gen[j]=flist[j]*gen[j]
    return gens


In [5]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= RandomForestClassifier(n_estimators=25).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [6]:
def BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    pheromone=list([T0]*dim)
    pher_diff=list([0]*dim)
    global_position=tuple([0]*dim)
    antslist=[]
    gens_dict = {}
    for i in range(n):
        gens=random_search(m,dim)
        mse=0
        for gen in gens:
            if tuple(gen) in gens_dict:
                score = gens_dict[tuple(gen)]
            else:
                score=evaluate(train_d,train_l,gen)
                gens_dict[tuple(gen)]=score
            global_best=score
            global_position=dc(gen)
            mse+=score
        mse=mse/m
        antslist.append(ants(gens,mse))
    antslist.sort(key=operator.attrgetter('mse'), reverse=True)
    klist=list(antslist[i] for i in range(k))
    mselist=list(antslist[i].mse for i in range(k))
    for j in range(len(klist)):
        diff=(max(mselist)-mselist[j])/(max(max(mselist)-mselist))
        flist=list([0]*dim)
        for gen in klist[j].gens:
            for a,x in enumerate(gen):
                if x==1:
                    pher_diff[a]=diff
                    flist[a]=1
                else:
                    pher_diff[a]=0
    
    for i in range(len(pheromone)):
        pheromone[i]=pheromone[i]*rho + pher_diff[i]
        
        
    for it in range(max_iter):
        m=m-p
        for i in range(n):
            gens=k_search(m,dim,flist,pheromone)
            mse=0
            for gen in gens:
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score
                global_best=score
                global_position=dc(gen)
                mse+=score
            mse=mse/m
            antslist.append(ants(gens,mse))
        antslist.sort(key=operator.attrgetter('mse'), reverse=True)
        klist=list(antslist[i] for i in range(k))
        mselist=list(antslist[i].mse for i in range(k))
        for j in range(len(klist)):
            diff=(max(mselist)-mselist[j])/(max(max(mselist)-mselist))
            flist=list([0]*dim)
            for gen in klist[j].gens:
                for i,x in enumerate(gen):
                    if x==1:
                        pher_diff[i]=diff
                        flist[i]=1
                    else:
                        pher_diff[i]=0
    
        for i in range(len(pheromone)):
            pheromone[i]=pheromone[i]*rho + pher_diff[i]

        
    return global_position,global_position.count(1)


In [7]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([RandomForestClassifier(n_estimators=25).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [8]:
with open("/home/shubbham28/Downloads/bc/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/bc/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [9]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.9734265734265735
1  000010001010001001000000000000  5  0.840559
2  011111101111011111011111101011  24  0.980420
3  111110111111011111111111111111  28  0.967832
4  110111111011111111111101111111  27  0.972028
5  011111000111110101111101010010  19  0.973427
6  010101011011111101010111100111  20  0.966434
7  010001000111000011010001100010  11  0.977622
8  111111011111111101101111111101  26  0.970629
9  000111111111111011111111111111  26  0.972028
10  100111100101111110100111011010  19  0.976224
11  111111111111111111111111101111  29  0.977622
12  000000000001100000000001000000  3  0.941259
13  010011100010000000000000000100  6  0.930070
14  111111111101111101111101101111  26  0.974825
15  001000000000000000000000000000  1  0.836364
16  000000010000000000001000000001  3  0.948252
17  111001111111111111011101111111  26  0.973427
18  111010000111000000000111000111  13  0.976224
19  000000010000000000010000000000  2  0.903497
20  110010000100110101101011111011  17  0.977622
Final:   0100110

In [8]:
with open("/home/shubbham28/Downloads/Arrhythmia/heart.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Arrhythmia/heart_output.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [9]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
print flist,fattr
final=np.argsort(flist)[::-1][:fattr]
print final
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.8647058823529411
1  1111101110011  10  0.879412
2  1001000100100  4  0.670588
3  1111111111101  12  0.785294
4  1110000110101  7  0.773529
5  0000010000000  1  0.529412
6  1111111011111  12  0.858824
7  1111011111111  12  0.864706
8  1010111000101  7  0.752941
9  1001101100111  8  0.805882
10  1111011111101  11  0.808824
11  0111011110011  9  0.852941
12  1101111101111  11  0.811765
13  1110011111011  10  0.873529
14  1000110101010  6  0.785294
15  1111111111111  13  0.861765
16  1111111111111  13  0.847059
17  1011011111111  11  0.850000
18  1101000000110  5  0.747059
19  1111110111111  12  0.852941
20  0000000000001  1  0.720588
[17, 13, 13, 14, 10, 14, 13, 15, 12, 11, 14, 13, 16] 8
[ 0 12  7 10  5  3 11  6]
Final:   1001011100111   8   0.796618    6676.0721
